# Imports JSON, travail sur les lemmes et typologie des variantes

In [ ]:
# On installe pie et on télécharge les modèles latins
!pip3 install pie-extended
!pie-extended download lasla

In [ ]:
# On importe plusieurs bibliothèques, dont celle de pie (pour l'annotations linguistique) et celle de collatex.
import collatex
import time
import json
import pie
import subprocess
import sys
sys.path.insert(1, 'utils/')
import utils.utils as utils

## Lemmatisation du texte

Commençons par lemmatiser notre texte; nous allons utiliser les modèles d'annotation produit par Thibault Clérice à partir des données du LASLA (https://github.com/chartes/deucalion-model-lasla). Créons une fonction simple qui appelle Pie.

In [ ]:
def lemmatize(path):
    device = "cpu"
    batch_size = 500
    # Le nom du modèle
    modele_lemmes = "models/lasla-plus-lemma.tar"
    modele_pos = "models/lasla-plus-pos.tar"
    modele = "models/model.tar"
    cmd = f' pie-extended tag lasla {path} --device {device} --batch_size {batch_size}'
    subprocess.run(cmd.split())
    print(f"Texte annoté enregistré sous {path.replace('.txt', '-pie.txt')}")

Nous n'avons maintenant qu'à appeler notre fonction sur les textes à annoter:

In [ ]:
lemmatize("Catullus/TEXT-Bodmer47-1.txt")
lemmatize("Catullus/TEXT-O1.txt")
lemmatize("Catullus/TEXT-G2.txt")

Cette étape peut être un peu longue; si elle ne fonctionne pas, les textes lemmatisés sont déjà disponibles à l'emplacement adéquat. Voyons la structure d'un des textes annotés:


In [ ]:
with open("Catullus/TEXT-G2-pie.txt", "r") as input_text:
    print(input_text.read())

Passons maintenant à l'étape suivante. Nous voulons pouvoir inclure les annotations linguistiques dans le processus de collation. Pour ce faire, il faut utiliser un format spécifique qu'est le JSON.

## Structures et avantages du format JSON pour collatex

Collatex demande une structure particulière si l'on veut travailler avec des données non formelles (image tirée de la documentation de l'outil)
![Données collatex](img/collatex_json.png)

Comme on le voit, chaque texte est présenté tokénisé, l'un après l'autre. Il peut contenir des données normalisées (c'est le cas pour l'entrée `n:cat`), qui seront celles prises en compte pour l'alignement. Nous allons donc produire la table pour collatex en utilisant d'abord les **formes** comme référence. Nous pouvons ajouter autant d'information que nécessaire sous ce format; en outre, nous pouvons ainsi aligner en utilisant les des formes normalisées pour améliorer l'alignement.

Commençons par importer nos textes lemmatisés, et par les convertir en listes. C'est le rôle de la fonction `import_annotated_data()`

In [ ]:
# On importe chacun des textes
bodmer_as_list = utils.import_annotated_data("Catullus/TEXT-Bodmer47-1-pie.txt")
O1_as_list = utils.import_annotated_data("Catullus/TEXT-O1-pie.txt")
G2_as_list = utils.import_annotated_data("Catullus/TEXT-G2-pie.txt")
dict_of_text = {"Bodmer47": bodmer_as_list, "O1": O1_as_list, "G2": G2_as_list}

Il faut maintenant convertir ces listes en un dictionnaire qui convienne à CollateX. La fonction `create_json_input_for_collatex()` s'en charge:

In [ ]:
json_input_forms = utils.create_json_input_for_collatex(dict_of_text, collate_on="forms")

Voyons ce que donne le dictionnaire ainsi créé:

In [ ]:
print(json_input_forms)

## Alignements sur les formes, sur les lemmes, sur lemmes+pos

Les tokens du dictionnaire contiennent toutes les informations dont nous aurons besoin par la suite: la forme `t`, la forme normalisée `n`, le lemme `lemma`, la partie du discours `pos`, la morphologie `morph`. On peut maintenant lancer la collation, en commençant par un **alignement sur les formes**.

In [ ]:
result_table_forms = collatex.collate(json_input_forms, output="html2", segmentation=False, near_match=True)

La table d'alignement sur les formes est de qualité moyenne, on y compte un certain nombre d'erreurs. Comment améliorer l'alignement ? On peut penser à améliorer la *normalisation* des données, en supprimant l'information graphique et grammaticale: c'est ce que fait la lemmatisation. Passons donc à un **alignement sur les lemmes**:

In [ ]:
json_input_lemmas = utils.create_json_input_for_collatex(dict_of_text, collate_on="lemmas")
result_table_lemmas = collatex.collate(json_input_lemmas, output="html2", segmentation=False, near_match=True)

Le résultat est meilleur: le début du texte est aligné de façon correcte, mais il reste quelques erreurs. Pouvons nous encore améliorer les résultats? Possiblement, en choisissant d'**aligner sur la concaténation du lemme et de la partie du discours**. De la sorte, en cas de divergence de lemme (variante lexicale), l'outil pourra toujours s'accrocher à la partie du discours, qui restera probablement inchangée.

In [ ]:
json_input_lemmas_pos = utils.create_json_input_for_collatex(dict_of_text, collate_on="lemmas+pos")
result_table_lemmas_pos = collatex.collate(json_input_lemmas_pos, output="html2", segmentation=False, near_match=True)

## Typologie des variantes

C'est cette dernière table d'alignement que nous allons choisir afin de travailler sur les lieux variants et le classement des variantes. Nous allons utiliser la sortie JSON (`output='json'`) proposée par Collatex.

In [ ]:
resultat_alignement_collatex = collatex.collate(json_input_lemmas_pos, output='json', segmentation=False, near_match=True)

Nous pouvons imprimer ce résultat, qui est difficilement lisible: il contient l'alignement des trois textes, l'un après l'autre.

In [ ]:
print(resultat_alignement_collatex)

Nous allons donc tâcher de travailler cette sortie pour classer les lieux variants. Pour ce faire il faut d'abord regrouper toutes les unités d'alignement (= chaque *token* ou mot aligné):

In [ ]:
simplified = utils.simplify_results(resultat_alignement_collatex)

On arrive ainsi au résultat suivant (exemple sur la dernière unité d'alignement)

In [ ]:
print(simplified[-1])

C'est avec ce format de données que nous allons maintenant travailler. La cellule suivante propose une fonction de classification sommaire qui sera ici utilisée.

In [ ]:
# Cette cellule charge les fonctions principales permettant d'analyser les variantes

def check_pos(locus):
    all_pos = [witness['pos'] for witness in locus]
    print(f"Vérifions la nature: {all_pos}")
    if all([pos == all_pos[0] for pos in all_pos[1:]]):
        print("La partie du discours est identique.")
        return {'pos': True}
    else:
        print("Une différence de nature semble apparaître: variante syntaxique ou grammaticale")
        return {'pos': False}


def check_morphology(locus):
    all_morph = [witness['morph'] for witness in locus]
    print(f"{' vs '.join(all_morph)}")
    if all([morph == all_morph[0] for morph in all_morph[1:]]):
        print("La morphologie est identique: variante graphique")
        return {'pos': True}
    else:
        print("Une différence de morphologie semble apparaître: variante syntaxique ou grammaticale")
        return {'pos': False}

def check_annotations(locus):
    all_lemmas = [witness['lemme'] for witness in locus]
    print(all_lemmas)
    all_lemmas_as_string = " | ".join(all_lemmas)
    print(f"Vérifions les lemmes: {all_lemmas_as_string}")
    if all([lemma == all_lemmas[0] for lemma in all_lemmas[1:]]):
        print("Les lemmes sont identiques.")
        return {**check_pos(locus), **{"lemmas": True}}
    else:
        print("Les lemmes sont distincts. Variante lexicale")
        return {"lemmas": False, "pos":"UNK"}


def analyse_lieux_variants(collatex_output):
    results = utils.simplify_results(collatex_output)
    # On crée une boucle sur chaque mot aligné
    for index, locus in enumerate(results):
        print(f"Unité d'alignement n°{index + 1}.")
        # On commence par comparer les formes
        print(f"Comparons les formes: {' | '.join([witness['forme'] if witness['forme'] != None else 'ø' for witness in locus])}")
        forme_base = locus[0]['forme']
        print(f"La forme base de la comparaison est: {forme_base}")
        # Si toutes les formes sont identiques entre elles, alors il n'y a pas de lieu variant.
        if all([witness['forme'] == forme_base for witness in locus]):
            print("Toutes les formes sont identiques, il n'y a pas de lieu variant.")
            
        # Au contraire, s'il y a une divergence formelle, il faut creuser pour voir si il s'agit d'une variante

        # Un cas possible est celui de l'omission d'un des témoins
        elif any([witness['forme'] == None for witness in locus]):
            all_forms = [witness['forme'] for witness in locus if witness['forme'] != None]
            all_forms_as_string = " | ".join(all_forms)
            print(f"On note une omission à cet endroit du texte. \nVérifions si les autres témoins concordent: {all_forms_as_string}")
            
            # Si les autres témoins concordent, il s'agit d'un lieu variant avec omission d'un témoin (ou plus) uniquement
            if all([form == all_forms[0] for form in all_forms[1:]]):
                print("Les autres témoins concordent. Omission")

            # Dans le cas inverse, il faut creuser pour voir s'il s'agit d'une variante
            else:
                print("Les autres témoins discordent dans leur forme")
                locus = [witness for witness in locus if witness['forme'] != None]
                # On va appeler une fonction qui vérifie d'abord si les lemmes concordent, puis si les parties du discours concordent.
                annotations_check = check_annotations(locus)
                # Si les lemmes et les parties du discours sont strictement identiques, nous avons une variante graphique
                if annotations_check['pos'] == True and annotations_check['lemmas'] == True:
                    print("Vérifions la morphologie:")
                    morph_check = check_morphology(locus)
        # Même processus que précédemment, mais sans omission.
        else:
            print("Les témoins discordent dans leur forme.")
            check_lemma = check_annotations(locus)
            if check_lemma['pos'] == True and check_lemma['lemmas'] == True:
                print("Vérifions la morphologie:")
                morph_check = check_morphology(locus)
            
    
        print("\n")

L'idée est de comparer successivement la forme, le lemme, la partie du discours et la morphologie des tokens alignés, unités d'alignement après unité d'alignement -- afin de classer les variantes. L'algorithme est fondé sur Camps, Jean-Baptiste, Lucence Ing, et Elena Spadini. « *Collating Medieval Vernacular Texts: Aligning Witnesses, Classifying Variants* », DH2019, Utrecht, 2019, dont est tiré le tableau suivant: 
![Collating](img/collating_2019.png)

In [ ]:
analyse_lieux_variants(resultat_alignement_collatex)

Comme on le voit, le processus est très sensible à la qualité de l'annotation et de la lemmatisation, qui est lui-même dépendant de la variabilité graphique des témoins; en l'occurrence, le modèle est ici peu performant car il a été entraîné sur des données issues d'éditions: les unités d'alignement 5 et 30 par exemple sont classées comme variantes lexicales, alors qu'elles ne sont que des variantes graphiques (les lemmes ne sont pas correctement attribués). La phase d'annotation lexico-grammaticale est donc fondamentale et les modèles d'annotation doivent être le plus précis possible. 

On pourrait aller plus loin en précisant la classification pour indiquer des variations dans la modalité, ou la flexion, comme le font Camps et al. 

Une étape ultérieure serait celle de l'intégration du sémantisme à l'étude des variantes, afin d'identifier les variantes discursives (et/atque dans l'exemple qui nous intéresse consiste en une variante lexicale qui n'est pas significative, par exemple). 
